# Analysis

### Imports

In [1]:
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

### Load data

In [2]:
train = pd.read_csv("../../data/home_default/train.csv")
test  = pd.read_csv("../../data/home_default/test.csv")

print("Shape of train:", train.shape)
print("Shape of test:",  test.shape)

Shape of train: (307511, 122)
Shape of test: (48744, 121)
